# AudioCraft, MusicGen Model Theory

## Abstract

**AudioCraft** is a free and open-source, high-quality audio and music generation framework, developed by [Meta](https://ai.meta.com). It concludes of three main tools: 
* MusicGen: Generate music from text prompts;
* AudioGen: Generate environmental sounds or sound effects from text prompts;
* EnCodec: Compresses and reconstructs audio with high fidelity [1].

**MusicGen** is a single-stage transformer LM (Language Model), operating with tokens (several streams of compressed discrete music representation). It is used to generate high-quality music (mono & stereo sound), with interpreting descriptive text-prompt [2].

## Introduction



## Sources

1. [AudioCraft, Meta Blog](https://ai.meta.com/blog/audiocraft-musicgen-audiogen-encodec-generative-ai-audio/)
2. [Copet, J., Kreuk, F. ,Gat, I., Remez, T., Kant, D., Synnaeve, G., Adi, Y., Défossez, A. (2024)  *Simple and Controllable Music Generation.*](https://doi.org/10.48550/arXiv.2306.05284)